# Pandas Series: trūkstami duomenys (missing data)

Šiame faile pateikiami `pandas.Series` pavyzdžiai, skirti trūkstamų duomenų (missing data) temai:
trūkstamų reikšmių identifikavimas ir tvarkymas (handling).
Pavyzdžiai orientuoti į duomenų analitikos situacijas ir pateikiami paprastai bei nuosekliai.

In [ ]:
import pandas as pd
import numpy as np

## 1. Kas laikoma trūkstamais duomenimis?

Pandas trūkstamas reikšmes dažniausiai žymi:
- `NaN` (skaitiniuose duomenyse)
- `None` (dažnai tekstiniuose, bet pandas dažnai konvertuoja į `NaN`/`<NA>`)
- `pd.NA` (pandas „nullable“ tipams)

Svarbu suprasti, kad `0` nėra trūkstama reikšmė, o validi skaitinė reikšmė.

In [12]:
s = pd.Series([10, None, 25, np.nan, 0, pd.NA, 40])
print(s)
print("dtype:", s.dtype)

0      10
1    None
2      25
3     NaN
4       0
5    <NA>
6      40
dtype: object
dtype: object


## 2. Trūkstamų reikšmių identifikavimas

Dažniausi metodai:
- `isna()` / `isnull()` – grąžina boolean `Series`, kur trūkstama reikšmė pažymėta `True`
- `notna()` / `notnull()` – priešingai, trūkstama pažymima `False`
- `sum()` – patogu suskaičiuoti trūkstamų kiekį (nes `True` laikomas 1)

In [13]:
print("isna():")
print(s.isna())

print("\nTrūkstamų reikšmių skaičius:", s.isna().sum())
print("Neturinčių trūkstamų reikšmių skaičius:", s.notna().sum())

isna():
0    False
1     True
2    False
3     True
4    False
5     True
6    False
dtype: bool

Trūkstamų reikšmių skaičius: 3
Neturinčių trūkstamų reikšmių skaičius: 4


### Trūkstamų reikšmių pozicijos

Kartais naudinga gauti indeksus, kur trūksta reikšmių.

In [ ]:
missing_idx = s[s.isna()].index
print("Indeksai su trūkstamomis reikšmėmis:", list(missing_idx))

## 3. Dažna analitinė situacija: trūkstamos reikšmės po konvertavimo

Kai reikšmės ateina kaip tekstas, o dalis jų nėra skaičiai, konvertuojant į skaičius atsiranda `NaN`.
Tam naudojama `pd.to_numeric(..., errors='coerce')`.

In [14]:
prices_raw = pd.Series(["10.5", "20.0", "not_available", "", "15.75", None])
prices = pd.to_numeric(prices_raw, errors="coerce")

print("prices_raw:")
print(prices_raw)
print("\nprices (po konvertavimo):")
print(prices)
print("\nTrūkstamų reikšmių skaičius:", prices.isna().sum())

prices_raw:
0             10.5
1             20.0
2    not_available
3                 
4            15.75
5             None
dtype: object

prices (po konvertavimo):
0    10.50
1    20.00
2      NaN
3      NaN
4    15.75
5      NaN
dtype: float64

Trūkstamų reikšmių skaičius: 3


## 4. Trūkstamų reikšmių tvarkymas (handling missing data)

Dažniausi keliai:
1) pašalinti trūkstamas (`dropna`)
2) užpildyti trūkstamas (`fillna`)
3) palikti trūkstamas ir skaičiavimuose remtis pandas elgsena (dažnai agregavimas ignoruoja `NaN`)

Sprendimas priklauso nuo reikšmės prasmės ir analizės tikslo.

### 4.1 `dropna()` – trūkstamų pašalinimas

`dropna()` pašalina įrašus su trūkstamomis reikšmėmis.
Tai tinka, kai trūkstamų yra mažai ir jų praradimas nepakeičia analizės išvadų.

In [16]:
%who

clean_drop	 daily_metric	 fill_median	 fill_zero	 inventory	 missing_idx	 np	 pd	 prices	 
prices_raw	 s	 sales	 status	 


In [17]:
clean_drop = prices.dropna()
print("Originalus ilgis:", len(prices))
print("Po dropna() ilgis:", len(clean_drop))
print(clean_drop)

Originalus ilgis: 6
Po dropna() ilgis: 3
0    10.50
1    20.00
4    15.75
dtype: float64


### 4.2 `fillna()` – trūkstamų užpildymas

`fillna()` leidžia užpildyti trūkstamas reikšmes pagal taisyklę:
- konkrečia reikšme (pvz., 0)
- statistika (pvz., vidurkiu, mediana)
- artimiausia reikšme (pvz., `ffill`, `bfill`)

Geroji praktika: prieš užpildymą aiškiai įvardyti, ką reiškia trūkstama reikšmė (pvz., „nežinoma“, „nebuvo pardavimų“, „neužfiksuota“).

In [18]:
# Užpildymas nuliu (tinka tik tada, kai 0 logiškai reiškia „nėra“)
fill_zero = prices.fillna(0)
print(fill_zero)

0    10.50
1    20.00
2     0.00
3     0.00
4    15.75
5     0.00
dtype: float64


In [19]:
# Užpildymas mediana (dažna praktika, kai duomenys turi outlier'ių)
fill_median = prices.fillna(prices.median())
print("Mediana:", prices.median())
print(fill_median)

Mediana: 15.75
0    10.50
1    20.00
2    15.75
3    15.75
4    15.75
5    15.75
dtype: float64


### 4.3 Užpildymas pagal artimiausią reikšmę: `ffill` ir `bfill`

- `ffill` (forward fill) užpildo trūkstamą reikšmę ankstesne žinoma reikšme.
- `bfill` (backward fill) užpildo trūkstamą reikšmę sekančia žinoma reikšme.

Tai dažnai naudojama laiko sekuose, kai rodiklis laikomas „galiojančiu“ iki kito atnaujinimo.

In [20]:
daily_metric = pd.Series(
    [100, None, None, 120, None, 130],
    index=pd.date_range("2025-01-01", periods=6, freq="D")
)

print("daily_metric:")
print(daily_metric)

print("\nffill:")
print(daily_metric.ffill())

print("\nbfill:")
print(daily_metric.bfill())

daily_metric:
2025-01-01    100.0
2025-01-02      NaN
2025-01-03      NaN
2025-01-04    120.0
2025-01-05      NaN
2025-01-06    130.0
Freq: D, dtype: float64

ffill:
2025-01-01    100.0
2025-01-02    100.0
2025-01-03    100.0
2025-01-04    120.0
2025-01-05    120.0
2025-01-06    130.0
Freq: D, dtype: float64

bfill:
2025-01-01    100.0
2025-01-02    120.0
2025-01-03    120.0
2025-01-04    120.0
2025-01-05    130.0
2025-01-06    130.0
Freq: D, dtype: float64


## 5. Trūkstamos reikšmės ir skaičiavimai

Pandas agregavimo metodai (`sum`, `mean`, ...) pagal nutylėjimą ignoruoja `NaN`.
Tai patogu, tačiau svarbu atskirti situacijas, kai:
- `NaN` reiškia „nėra duomenų“ (ignoravimas dažnai tinka)
- `NaN` reiškia „0“ (ignoravimas netinka)

Tokiu atveju reikia sąmoningai pasirinkti `fillna(0)` arba kitą taisyklę.

In [21]:
sales = pd.Series([10, 12, np.nan, 9, np.nan, 15])

print("sales:")
print(sales)

print("\nSum (ignoruojant NaN):", sales.sum())
print("Mean (ignoruojant NaN):", round(sales.mean(), 2))

print("\nSum su fillna(0):", sales.fillna(0).sum())
print("Mean su fillna(0):", round(sales.fillna(0).mean(), 2))

sales:
0    10.0
1    12.0
2     NaN
3     9.0
4     NaN
5    15.0
dtype: float64

Sum (ignoruojant NaN): 46.0
Mean (ignoruojant NaN): 11.5

Sum su fillna(0): 46.0
Mean su fillna(0): 7.67


## 6. Trūkstamų reikšmių interpretacija: paprasta klasifikacija

Kartais naudinga pažymėti, ar reikšmė yra trūkstama, ir turėti atskirą „kokybės“ indikatorių.
Tokiais atvejais naudojamas `isna()` ir `np.where`.

In [22]:
inventory = pd.Series([25, None, 0, 12, None, 7], index=["P01", "P02", "P03", "P04", "P05", "P06"])

status = np.where(inventory.isna(), "Missing", "Available")

print("inventory:")
print(inventory)
print("\nstatus:")
print(pd.Series(status, index=inventory.index))

inventory:
P01    25.0
P02     NaN
P03     0.0
P04    12.0
P05     NaN
P06     7.0
dtype: float64

status:
P01    Available
P02      Missing
P03    Available
P04    Available
P05      Missing
P06    Available
dtype: object


## 7. Dažnos klaidos ir gerosios praktikos (santrauka)

Dažnos klaidos:
- `0` laikymas trūkstama reikšme (0 dažnai reiškia realų faktą: „nebuvo pardavimų“, „nėra likučio“).
- `dropna()` naudojimas neįvertinus, kiek eilučių prarandama ir ar tai nesukelia šališkumo.
- `fillna(0)` naudojimas, kai 0 nėra prasminga reikšmė (pvz., kainai ar reitingui).
- Trūkstamų reikšmių „paslėpimas“ nepasižymint, kad jos buvo užpildytos.

Gerosios praktikos:
- Pirma identifikuoti trūkstamų kiekį (`isna().sum()`), tik tada spręsti, ką daryti.
- Aiškiai apibrėžti, ką reiškia trūkstama reikšmė konkrečiame rodiklyje.
- Jei užpildoma, naudoti logišką taisyklę (0, mediana, ffill/bfill) ir, kai reikia, išsaugoti papildomą indikatorių.
- Patikrinti rezultatą po tvarkymo (pvz., `isna().sum()` prieš ir po).

## 8. Trumpa santrauka

- Trūkstamos reikšmės identifikuojamos su `isna()` ir `notna()`.
- `dropna()` pašalina trūkstamas, `fillna()` leidžia jas užpildyti.
- `ffill` ir `bfill` dažnai naudojami laiko sekoms.
- Agregavimas dažniausiai ignoruoja `NaN`, todėl reikia sąmoningai pasirinkti taisyklę, kai `NaN` reikšmė turi verslo prasmę.